In [1]:
!nvidia-smi

Thu Mar 27 17:04:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 46.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 22.9 MB/s eta 0:00:00


In [3]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.47.0
Uninstalling transformers-4.47.0:
  Successfully uninstalled transformers-4.47.0
Found existing installation: accelerate 1.2.1
Uninstalling accelerate-1.2.1:
  Successfully uninstalled accelerate-1.2.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 30.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 18.5 MB/s eta 0:00:00


## Importing Libraries


In [4]:
import os
import sys 
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer,TFAutoModelForSeq2SeqLM,DataCollatorForSeq2Seq,AdamWeightDecay

## Loading Dataset

In [5]:
dataset = load_dataset('cfilt/iitb-english-hindi')

README.md:   0%|          | 0.00/3.14k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [7]:
dataset['train'][:15]

{'translation': [{'en': 'Give your application an accessibility workout',
   'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'},
  {'en': 'Accerciser Accessibility Explorer',
   'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'},
  {'en': 'The default plugin layout for the bottom panel',
   'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'},
  {'en': 'The default plugin layout for the top panel',
   'hi': 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका'},
  {'en': 'A list of plugins that are disabled by default',
   'hi': 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है'},
  {'en': 'Highlight duration', 'hi': 'अवधि को हाइलाइट रकें'},
  {'en': 'The duration of the highlight box when selecting accessible nodes',
   'hi': 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि'},
  {'en': 'Highlight border color',
   'hi': 'सीमांत (बोर्डर) के रंग को हाइलाइट करें'},
  {'en': 'The color and opacity of the highlight border.',
   'hi': 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। '},
  {'en

In [8]:
model= 'Xenova/opus-mt-en-hi'

In [9]:
tokenizer= AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.68M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [10]:
tokenizer

MarianTokenizer(name_or_path='Xenova/opus-mt-en-hi', vocab_size=61950, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	61949: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [11]:
tokenizer("My name is Vishal")

{'input_ids': [633, 300, 23, 3654, 5511, 916, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [12]:
tokenizer(['Hello My name is Vishal','I love to cook chicken today'])

{'input_ids': [[12110, 633, 300, 23, 3654, 5511, 916, 0], [56, 362, 7, 18814, 28985, 765, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}

In [13]:
tokenizer(['Hello','My','name','is','Vishal'])

{'input_ids': [[12110, 0], [633, 0], [300, 0], [23, 0], [3654, 5511, 916, 0]], 'attention_mask': [[1, 1], [1, 1], [1, 1], [1, 1], [1, 1, 1, 1]]}

In [14]:
tokenizer(['मेरा नाम विशाल है।'])

{'input_ids': [[44, 1056, 174, 428, 260, 44, 314, 260, 1056, 44, 1, 260, 800, 44, 1, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [15]:
tokenizer(text_target=['मेरा नाम विशाल है।'])

{'input_ids': [[500, 179, 7767, 5, 40, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]}

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [17]:
def data_preprocess(sentense):
    inputs = [ex['en'] for ex in sentense['translation']]
    targets = [ex['hi'] for ex in sentense['translation']]

    #tokenized inputs
    encoded_inputs = tokenizer(inputs,max_length=256,truncation=True)

    encoded_targets = tokenizer(text_target=targets,max_length=256,truncation=True)

    encoded_inputs['labels'] = encoded_targets['input_ids']
    return encoded_inputs

In [18]:
tokenized_data = dataset.map(data_preprocess,batched=True)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [19]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2507
    })
})

In [153]:
model = TFAutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-en-hi')

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [154]:
batch_size=2
learning_rate=0.00001
weight_decay=0.001
num_train_epoch=1


In [155]:
data_collator = DataCollatorForSeq2Seq(tokenizer,model=model, return_tensors="tf")

In [156]:
training_dataset = model.prepare_tf_dataset(
    tokenized_data['test'],batch_size=batch_size,shuffle=True,collate_fn=data_collator)

In [157]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_data['validation'],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator
)

In [158]:
optimizer=AdamWeightDecay(learning_rate,weight_decay)

In [159]:
model.compile(optimizer)

In [160]:
model.summary()

Model: "tf_marian_mt_model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFMarianMainLayer)   multiple                  76381184  
                                                                 
 final_logits_bias (BiasLay  multiple                  61950     
 er)                                                             
                                                                 
Total params: 76443134 (291.61 MB)
Trainable params: 76381184 (291.37 MB)
Non-trainable params: 61950 (241.99 KB)
_________________________________________________________________


In [ ]:
model.fit(training_dataset,validation_data=validation_dataset,epochs=num_train_epoch)

 906/1253 [====================>.........] - ETA: 26s - loss: 3.6960

In [ ]:
model.save_pretrained('/kaggle/working/model/')

In [ ]:
input_text = "my name is vishal"

In [ ]:
tokenized = tokenizer([input_text],return_tensors='np')

In [ ]:
tokenized

In [ ]:
out = model.generate(**tokenized,max_length=256).numpy()

In [ ]:
out[0]

In [ ]:
tokenizer.decode(out[0],skip_special_tokens=True)